
# Spot resources Analytics

Here we perform some initial process and analysis on the dataset.

---




With static dataset, e.g. load the grabbed data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
import time
import dateutil.parser

/home/shwsun/py2_kernel/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
# The filtered file make sure to use the fi
#filename = 'data/aws-recent-filtered.csv'
filename = './data/subset.csv'

In [3]:
start = time.time()
# parse the data file and extra the results
df = pd.read_csv(filename)
print(time.time()-start)
df.head(3)

1.2520980835


,TimeStamp,AvailabilityZone,InstanceType,SpotPrice
0,2016-12-12 10:46:15,ap-northeast-2a,m4.large,0.0353
1,2016-12-12 10:46:05,ap-northeast-2a,m4.large,0.0351
2,2016-12-12 10:45:05,ap-northeast-2c,m4.large,0.0244


In [4]:
df = df.dropna()
df = df[df.AvailabilityZone != "AvailabilityZone"]
df['TimeStamp'] = pd.to_datetime(df.TimeStamp)
df.index = df.TimeStamp
df = df.drop('TimeStamp', axis=1)
df= df.sort_index()
df.head(3)

,AvailabilityZone,InstanceType,SpotPrice
TimeStamp,,,
2016-10-11 11:09:34,ap-northeast-2c,i2.4xlarge,1.136
2016-10-11 11:09:34,ap-northeast-2a,i2.4xlarge,1.136
2016-10-11 11:10:00,ap-northeast-2a,i2.2xlarge,0.568


----------

In [5]:
print (len(df))
print (df['InstanceType'].unique())
print (df['AvailabilityZone'].unique())
print (df['SpotPrice'].unique())


1496361
['i2.4xlarge' 'i2.2xlarge' 'm4.16xlarge' 'x1.32xlarge' 'x1.16xlarge'
 'd2.8xlarge' 'r3.large' 'r3.4xlarge' 'r3.8xlarge' 'd2.xlarge' 'i2.8xlarge'
 'c4.2xlarge' 'i2.xlarge' 'd2.2xlarge' 'r3.2xlarge' 'c4.4xlarge'
 'd2.4xlarge' 'r3.xlarge' 'm4.10xlarge' 'm4.2xlarge' 'c4.xlarge'
 'm4.xlarge' 'm4.large' 'c4.large' 'c4.8xlarge' 'm4.4xlarge']
['ap-northeast-2c' 'ap-northeast-2a']
[ 1.136   0.568   3.472  ...,  2.2382  2.2814  2.3162]


## Hypothesis #2

For each machine type there exists a region that is more favorable to use, as the market volatility is very low and the prices tend to stay cheaper than the other regions.

With in proving this hypothesis users will be able to find the best region they should be bidding in, as long as latency is not an issue for them.

Data Science tools & Techniques: We can use clustering and classification methods.


In [6]:
def corrGraph(title, df):
    corr_df = df.corr()
    mask = np.zeros_like(corr_df)
    mask[np.triu_indices_from(mask)] = True
    seaborn.heatmap(corr_df, cmap='RdYlGn_r', vmax=1.0, vmin=-1.0 , mask = mask, linewidths=2.5)
    plt.yticks(rotation=0) 
    plt.title(title)
    plt.xticks(rotation=90) 
    plt.show()


In [7]:
# Some info about the data
df = df.truncate(before='2016-10-13 00:00:00', after='2016-12-11 00:00:00')
print (df.index.min())
print (df.index.max())
print(df.index.max()- df.index.min()) 

df.head(3)

2016-10-13 00:00:13
2016-12-11 00:00:00
58 days 23:59:47


,AvailabilityZone,InstanceType,SpotPrice
TimeStamp,,,
2016-10-13 00:00:13,ap-northeast-2c,c4.8xlarge,1.8506
2016-10-13 00:00:13,ap-northeast-2a,m4.large,0.0193
2016-10-13 00:00:14,ap-northeast-2a,r3.2xlarge,0.0838


In [8]:
depa = df.groupby(['AvailabilityZone', 'InstanceType'])

In [9]:
#depa = awsResampler(df)
# Initialize dictionary of all combos of  empty dfs we want to graph and corr
zonedfs={}
typedfs={}
for item in df['InstanceType'].unique():
    typedfs.update({item: pd.DataFrame()})
for item in df['AvailabilityZone'].unique():
    zonedfs.update({item: pd.DataFrame()})

#Fill zonedfs with dataframes of all machines in that zone pricing
for name, group in depa:
    #We have to create the data frame by merging the rows and the first run we need our first row
    if zonedfs[name[0]].empty:
        #RESAMPLE DATA HOURLY getting the avg of the hour
        zonedfs[name[0]] = group
        zonedfs[name[0]] = zonedfs[name[0]].resample('H').mean()
        zonedfs[name[0]] = zonedfs[name[0]].fillna(method="ffill")
        #zonedfs[name[0]] = zonedfs[name[0]].drop('InstanceType', axis=1).drop(['AvailabilityZone'],axis=1)
        #Remove index so that you can merge
        zonedfs[name[0]] = zonedfs[name[0]].reset_index()
        #print(zonedfs[name[0]].head(40) )
        zonedfs[name[0]].rename(columns = {'SpotPrice':name[1]}, inplace = True)

    else:
        #RESAMPLE DATA HOURLY getting the avg of the hour
        group1 = group.resample('H').mean()
        group1 = group1.fillna(method="ffill")
        #print(zonedfs[name[0]].head(20) )
        #group1 = group.drop('InstanceType', axis=1).drop(['AvailabilityZone'],axis=1)
        group1.rename(columns = {'SpotPrice':name[1]}, inplace = True)    
        #Remove index so that you can merge
        group1 = group1.reset_index()
        #print(group1.head(20))
        zonedfs[name[0]] = zonedfs[name[0]].merge(group1,how='right')


#Fill typedfs with dataframes of all machines in that zone pricing
for name, group in depa:
    if typedfs[name[1]].empty:
        typedfs[name[1]] = group
        typedfs[name[1]] = typedfs[name[1]].resample('H').mean()
        typedfs[name[1]] = typedfs[name[1]].fillna(method="ffill")
        #typedfs[name[1]] = typedfs[name[1]].drop('InstanceType', axis=1).drop(['AvailabilityZone'],axis=1)
        typedfs[name[1]].rename(columns = {'SpotPrice':name[0]}, inplace = True)
        typedfs[name[1]] = typedfs[name[1]].reset_index()
    else:
        group1 = group.resample('H').mean()
        group1 = group1.fillna(method="ffill")
        #group1 = group.drop('InstanceType', axis=1).drop(['AvailabilityZone'],axis=1)
        group1.rename(columns = {'SpotPrice':name[0]}, inplace = True)
        group1 = group1.reset_index()
        typedfs[name[1]] = typedfs[name[1]].merge(group1,how='right')


In [23]:
#for key in typedfs:
#    print(typedfs[key])
    
type(typedfs)
df = typedfs["c4.2xlarge"]
df.head(10)
#print(typedfs)
#s = pd.Series(data, index=index)
#df.index
#df.columns
#s = df['c4.2xlarge'].values
#s = list(df.values)

array = []
def populate_array(df):
    for c in df.columns:
        #print(c)
        #if c != 'TimeStamp':
        xs = df[c].values
        array.append(xs.tolist())
print("finished")

for x in [ 'd2.2xlarge', 'r3.2xlarge', 'c4.4xlarge', 
 'd2.4xlarge', 'r3.xlarge', 'm4.10xlarge', 'm4.2xlarge', 'c4.xlarge', 
 'm4.xlarge', 'm4.large', 'c4.large', 'c4.8xlarge', 'm4.4xlarge']:
    populate_array(typedfs[x])
    


finished


In [24]:
len(array[23])
df = pd.DataFrame(array)
df
df.to_csv("vmarrays.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416
0,1.476317e+18,1.476320e+18,1.476324e+18,1.476328e+18,1.476331e+18,1.476335e+18,1.476338e+18,1.476342e+18,1.476346e+18,1.476349e+18,...,1.481382e+18,1.481386e+18,1.481389e+18,1.481393e+18,1.481396e+18,1.481400e+18,1.481404e+18,1.481407e+18,1.481411e+18,NaN
1,2.364273e-01,2.088333e-01,2.120308e-01,2.118571e-01,2.143889e-01,2.089091e-01,2.133778e-01,2.080300e-01,2.414500e-01,2.260083e-01,...,1.791500e-01,1.828182e-01,1.929143e-01,1.825333e-01,1.910000e-01,2.643500e-01,1.806000e-01,1.786286e-01,1.791500e-01,NaN
2,2.416500e-01,2.291500e-01,2.337769e-01,2.416500e-01,2.546455e-01,2.236818e-01,2.668455e-01,2.224667e-01,2.692143e-01,2.224667e-01,...,2.462500e-01,2.462500e-01,2.462500e-01,2.462500e-01,2.572857e-01,2.996250e-01,2.462500e-01,2.462500e-01,2.462500e-01,NaN
3,1.476320e+18,1.476324e+18,1.476328e+18,1.476331e+18,1.476335e+18,1.476338e+18,1.476342e+18,1.476346e+18,1.476349e+18,1.476353e+18,...,1.481386e+18,1.481389e+18,1.481393e+18,1.481396e+18,1.481400e+18,1.481404e+18,1.481407e+18,1.481411e+18,NaN,NaN
4,8.400000e-02,8.400000e-02,8.445000e-02,8.395000e-02,8.446000e-02,8.390000e-02,8.455000e-02,8.396667e-02,8.478333e-02,9.986667e-02,...,8.800909e-02,9.090556e-02,8.768947e-02,1.029409e-01,1.084760e-01,8.391500e-02,8.516842e-02,8.713043e-02,NaN,NaN
5,8.738571e-02,8.658000e-02,8.705000e-02,8.659000e-02,8.705000e-02,8.801000e-02,8.705000e-02,1.030333e-01,1.030333e-01,1.030333e-01,...,1.116050e-01,9.747857e-02,1.036250e-01,1.082063e-01,1.039957e-01,8.755625e-02,8.468462e-02,8.868824e-02,NaN,NaN
6,1.476317e+18,1.476320e+18,1.476324e+18,1.476328e+18,1.476331e+18,1.476335e+18,1.476338e+18,1.476342e+18,1.476346e+18,1.476349e+18,...,1.481382e+18,1.481386e+18,1.481389e+18,1.481393e+18,1.481396e+18,1.481400e+18,1.481404e+18,1.481407e+18,1.481411e+18,NaN
7,1.063500e-01,1.069571e-01,1.064667e-01,1.066500e-01,1.061500e-01,1.061500e-01,1.063000e-01,1.063500e-01,1.044333e-01,1.008250e-01,...,2.300300e-01,1.136808e-01,9.935000e-02,9.940000e-02,9.910000e-02,9.942500e-02,1.045286e-01,2.458200e-01,9.926667e-02,NaN
8,9.875000e-02,9.925000e-02,9.870000e-02,9.915000e-02,9.880000e-02,9.900000e-02,9.880000e-02,9.880000e-02,9.894000e-02,9.893333e-02,...,1.833000e-01,1.134000e-01,1.085750e-01,1.064500e-01,1.111111e-01,1.272667e-01,1.345143e-01,3.028000e-01,1.278222e-01,NaN
9,1.476317e+18,1.476320e+18,1.476324e+18,1.476328e+18,1.476331e+18,1.476335e+18,1.476338e+18,1.476342e+18,1.476346e+18,1.476349e+18,...,1.481382e+18,1.481386e+18,1.481389e+18,1.481393e+18,1.481396e+18,1.481400e+18,1.481404e+18,1.481407e+18,1.481411e+18,NaN


## Correlations of timeseries

--------
### AVGS, MINS, MAXs vs REAL Prices

### Reference:

[pattern matching over time series data](http://stats.stackexchange.com/questions/136091/sequential-pattern-matching-in-time-series-data)